### Test of data instantiation

In [ ]:
import torch
from torch.utils.data import DataLoader 
import numpy as np
import matplotlib.pyplot as plt
import os
from torchmetrics import FBetaScore, Accuracy
import torchvision.transforms as T
from torchvision.transforms import functional as F

import src.dfmaker as dfmaker
import src.BWDatasets as BWDatasets

from paddleocr import PaddleOCR
import paddle


### Ensure Reproducibility

In [ ]:
seed = 115
generator = torch.Generator()
generator.manual_seed(seed)
np_generator = np.random.default_rng(seed=seed)

### Activate CUDA

In [ ]:
device = torch.device('cpu')

# Make sure the notebook is deterministic if training on gpu
if torch.cuda.is_available():
    device = torch.device('cuda')
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False

print(f'Cuda availability is: {torch.cuda.is_available()}')  # Returns True if a GPU is available
print(f"Training on device {device}.")

In [ ]:
def find_width(height):
    return int(1.7*height)

In [ ]:
# Lyngøy
label_path_lyng = "dataset/datasets/lyngoy/ringcodes.csv"
image_path_lyng = "dataset/datasets/lyngoy/images"
bb_path_lyng = "dataset/datasets/lyngoy/labels"

# RF
label_path_rf = "dataset/datasets/rf/ringcodes.csv"
image_path_rf = "dataset/datasets/rf/images"
bb_path_rf = "dataset/datasets/rf/labels"

# Ringmerkingno
label_path_rno = "dataset/datasets/ringmerkingno/ringcodes.csv"
image_path_rno = "dataset/datasets/ringmerkingno/images"
bb_path_rno = "dataset/datasets/ringmerkingno/labels"

height = 112
max_n_single = None
max_n_all = None
transform = T.Compose([
    T.Lambda(lambda x: F.rotate(x, -90, expand=True)),
    T.Resize((height, find_width(height)))
])

df_lyng = dfmaker.make_dataframe(labels_path=label_path_lyng, img_path=image_path_lyng, bb_path=bb_path_lyng, max_n=max_n_single)
df_rf = dfmaker.make_dataframe(labels_path=label_path_rf, img_path=image_path_rf, bb_path=bb_path_rf, max_n=max_n_single)
df_rno = dfmaker.make_dataframe(labels_path=label_path_rno, img_path=image_path_rno, bb_path=bb_path_rno, max_n=max_n_single)
complete_df = dfmaker.combine_dfs([df_lyng, df_rf, df_rno])

exp_dataset = BWDatasets.TrainDataSet(df=complete_df, transform=transform, max_n=max_n_all)
exp_loader = DataLoader(dataset=exp_dataset, batch_size=64, shuffle=False, generator=generator)

In [ ]:
#stop

In [ ]:
def draw_image(img):
    img_uint8 = BWDatasets.tensor_to_numpy(img)
    plt.figure(figsize=(12,12))
    plt.imshow(img_uint8)
    plt.axis("off")
    plt.show()

In [ ]:
data = exp_dataset[0]

draw_image(data["ocr_image"])
draw_image(data["image"])


In [ ]:
#stop



In [ ]:
paddle.device.set_device("gpu:0")
ocr = PaddleOCR(lang="en", 
           use_doc_orientation_classify=True, 
           use_doc_unwarping=True, 
           use_textline_orientation=True, device='gpu')


In [ ]:
def prediction(dataloader):
    preds = []
    targets = []
    count = 0
    for data in dataloader:
        count +=1
        print(f'Batch {count} of {len(dataloader)}')
        images = BWDatasets.tensor_to_numpy(data["ocr_image"])
        labels = data["label"][0]
        for i, image in enumerate(images):
            
            result = ocr.predict(image)

            pred = result[0]["rec_texts"]

            if not pred:
                pred = ['Not Found']

            preds.append(pred[0])

            label = labels[i]
            
            targets.append(label)

    return preds, targets


In [ ]:
def evaluate_model(preds, labels):
    # print(f'preds: {preds}')
    # print(f'labels: {labels}')
    
    # Boolean mask for the predictions and the labels
    # Prediction = 1 if it is correct, 0 otherwise
    # Labels are always = to 1
    bin_preds = [int(pred == label) for pred, label in zip(preds, labels)]
    bin_labels = [1]*len(labels)

    preds_tensor, labels_tensor = torch.tensor(bin_preds), torch.tensor(bin_labels)
    fbeta = FBetaScore(task='binary', beta=0.5)
    acc = Accuracy(task='binary')

    print(f'Model F1 Score: {fbeta(preds_tensor, labels_tensor)}')
    print(f'Model Accuracy: {acc(preds_tensor, labels_tensor)}')
    # print(f'preds: {preds_tensor}')
    # print(f'labesl: {labels_tensor}')

    return acc(preds_tensor, labels_tensor)

In [ ]:

exp_loader = DataLoader(dataset=exp_dataset, batch_size=64, shuffle=False, generator=generator)


In [ ]:
height1 = 32
height2 = 64
height3 = 200

SIZES = { 
        "Height 32": (height1, find_width(height1)),
        "Height 64": (height2, find_width(height2)),
        "Height 200": (height3, find_width(height3))
        }

predictions = {}

for sizestr, size in SIZES.items():
    print(f'Size: {sizestr} ')

    newtrans = T.Compose([
    T.Lambda(lambda x: F.rotate(x, -90, expand=True)),
    T.Resize((size[0], size[1]))
    ])
    exp_dataset.transform = newtrans
    
    exp_loader = DataLoader(dataset=exp_dataset, batch_size=64, shuffle=False, generator=generator)

    preds, labels = prediction(exp_loader)

    result = evaluate_model(preds=preds, labels=labels)

    predictions[sizestr] = result
    


In [ ]:
print(predictions)